In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
plates = pd.DataFrame({
    'data_path': [base_path/'2024.11.07_exp117.4'/'export'/f'plate{n}' for n in range(1,4)],
    'yaml_path': [base_path/'2024.11.07_exp117.4'/'export'/'wells.yaml']*3,
    'biorep': range(1,4)
})
output_path = rd.rootdir/'output'/'lenti_therapeutic-titer'
cache_path = output_path/'lenti_therapeutic-titer.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

### Calculate titer
`rd.flow.moi` takes:

1. A DataFrame with the following columns:

    - condition
    - replicate
    - starting_cell_count
    - scaling (dilution factor relative to max_virus)
    - max_virus

2. Information to gate infected cells

    - color_column_name (channel to gate on)
    - color_column_cutoff (gate)

3. Optional parameters

    - output_path (where to save the plots)
    - summary_method (mean/median of replicates)

In [ ]:
# Create columns for rd.flow.moi
data['condition'] = data['construct']
data['starting_cell_count'] = 2e4
data['max_virus'] = 4

In [ ]:
# Draw gates on uninfected population
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['condition']=='UI'].groupby(['biorep'])[channel].apply(lambda x: x.quantile(0.995))
gates.reset_index(inplace=True)

In [ ]:
display(gates)

In [ ]:
plot_df = data[(data['condition']=='UI')]
x = 'mGL-A'
y = 'mRuby2-A'
g = sns.displot(data=plot_df, x=x, y=y, kind='kde', col='biorep',
                log_scale=True, hue='construct', legend=False, color=base.colors['gray'],
                common_norm=False, fill=False, levels=7, facet_kws=dict(margin_titles=True))

for biorep, ax in g.axes_dict.items():
    gate = gates[(gates['biorep']==biorep)]
    if gate.empty: continue
    ax.axvline(gate[x].values[0], color='black', zorder=0)
    ax.axhline(gate[y].values[0], color='black', zorder=0)

g.figure.savefig(rd.outfile(output_path/'kde_mGL-mRuby2_UI-gates.png'))

In [ ]:
for biorep, group in data.groupby('biorep'):

    plot_df = group[(group['scaling']==1)].groupby('condition').sample(1000)

    g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', col='condition', col_wrap=4, kind='kde',
                    log_scale=True, common_norm=False, fill=False, levels=7,
                    hue='construct')
    
    gate = gates[(gates['biorep']==biorep)]
    if gate.empty: continue

    for _, ax in g.axes_dict.items():
        ax.axvline(gate[x].values[0], color='black', zorder=0)
        ax.axhline(gate[y].values[0], color='black', zorder=0)

    g.figure.savefig(rd.outfile(output_path/f'kde_mGL-mRuby2_by-construct-with-gates_biorep{biorep}.png'))

In [ ]:
# Add marker/output metadata
data['marker'] = data['mGL-A']
data.loc[data['construct'].isin(['RC809','RC810','RC811']), 'marker'] = data.loc[data['construct'].isin(['RC809','RC810','RC811']), 'mRuby2-A']

data['output'] = data['mRuby2-A']
data.loc[data['construct'].isin(['RC809','RC810','RC811']), 'output'] = data.loc[data['construct'].isin(['RC809','RC810','RC811']), 'mGL-A']

In [ ]:
# Categorize cells into quadrants based on two gates
# Possible values:
#   0 = double negative
#   1 = x-positive
#   2 = y-positive
#   3 = double positive
def get_quadrant(df, x, y, gates):
    gate_x = gates.loc[(gates['biorep']==df['biorep'].values[0]), x]
    gate_y = gates.loc[(gates['biorep']==df['biorep'].values[0]), y]
    df['x'] = data[x] > gate_x.values[0]
    df['y'] = data[y] > gate_y.values[0]
    df['quadrant'] = df['x'].astype(int) + df['y'].astype(int)*2
    return df

x = 'mGL-A'
y = 'mRuby2-A'
data = data.groupby(['biorep'])[data.columns].apply(lambda df: get_quadrant(df, x, y, gates))
data.reset_index(drop=True, inplace=True)
display(data)

In [ ]:
# Gate on marker-positive cells to calculate titer
#   RC806-8 on mGL: x-positive (1) or double-positive (3)
#   RC809-11 on mRuby2: y-positive (2) or double-positive (3)

data['gated'] = data['quadrant'] > 1.5
data.loc[data['construct'].isin(['RC806','RC807','RC808']), 'gated'] = data.loc[data['construct'].isin(['RC806','RC807','RC808']), 'quadrant'].isin([1,3])
display(data)

In [ ]:
# Calculate titer
df_titer = data.groupby(['biorep'])[data.columns].apply(lambda df: rd.flow.moi(df, 'gated', 0.5))
df_titer.reset_index(inplace=True)
display(df_titer)